In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

from sklearn.metrics import accuracy_score,precision_score,recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers,losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

In [ ]:
(x_train,_),(x_test,_)=fashion_mnist.load_data()
x_train=x_train.astype('float32')/255
x_test=x_test.astype('float32')/255
print (x_train.shape)
print (x_test.shape)

In [ ]:
fig,ax=plt.subplots(2,7)
i=0
j=0
for el in x_train[0:14]:
    ax[j][i].imshow(el, cmap=plt.get_cmap('gray'))
    i+=1
    if(i==7):
        j+=1
        i=0

In [ ]:
#autoencoders: extract features from a sample space, and then decode to reproduce the results
#here latent dimension is 64
latent_dim=64
input_im=tf.keras.Input(shape=(28,28,1))
encoder=tf.keras.Sequential([layers.Flatten(),layers.Dense(latent_dim,activation='relu')])
decoder=tf.keras.Sequential([layers.Dense(784,activation='sigmoid'),layers.Reshape((28,28))])
def autoenc(x_arr):
    feature=encoder(x_arr)
    output=decoder(feature)
    return output
model=Model(input_im,autoenc(input_im))
model.compile(optimizer='adam', loss=losses.MeanSquaredError())


In [ ]:
conv_encoder = tf.keras.Sequential([layers.Input(shape=(28, 28, 1)),layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
    layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)])

conv_decoder = tf.keras.Sequential([layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
    layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
    layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')])

def conv_autoenc(x):
    encoded = conv_encoder(x)
    decoded = conv_decoder(encoded)
    return decoded

conv_model=Model(input_im,conv_autoenc(input_im))
conv_model.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [ ]:
#model.summary()
conv_model.summary()

In [ ]:
conv_model.fit(x_train, x_train,epochs=10,shuffle=True,validation_data=(x_test, x_test))

In [ ]:
encoded_imgs = conv_encoder(x_test.reshape(10000,28,28,1)).numpy()
decoded_imgs = conv_decoder(encoded_imgs).numpy()

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
  # display original
  ax = plt.subplot(2, n, i + 1)
  plt.imshow(x_test[i])
  plt.title("original")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)

  # display reconstruction
  ax = plt.subplot(2, n, i + 1 + n)
  plt.imshow(decoded_imgs[i])
  plt.title("reconstructed")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
print(encoded_imgs)
for i in range(n):
  # display original
  ax = plt.subplot(2, n, i + 1)
  plt.plot(encoded_imgs[i])
  plt.title("features")
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)